In [1]:
import os 
#set working directory
os.chdir("../")

In [2]:
# show current working directory
print(os.getcwd())

/Users/bguenes/Documents/Uni/Master/Masterarbeit


In [46]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
from scipy.stats import norm
from scipy.stats import multivariate_normal
from tqdm import tqdm

from Chempy.parameter import ModelParameters

import sbi.utils as utils

import torch
from torch.distributions.normal import Normal
from torch.distributions.uniform import Uniform

import time as t
import pickle

import _multiprocessing as mp

In [4]:
# ----- Load the data -----
a = ModelParameters()
labels = [a.to_optimize[i] for i in range(len(a.to_optimize))] + ['time']
priors = torch.tensor([[a.priors[opt][0], a.priors[opt][1]] for opt in a.to_optimize])

elements = a.elements_to_trace

In [5]:
# ----- Load posterior -----
with open('master_thesis/data/posterior_sbi.pickle', 'rb') as f:
    posterior = pickle.load(f)

In [6]:
# ----- Load the NN -----
class Model_Torch(torch.nn.Module):
    def __init__(self):
        super(Model_Torch, self).__init__()
        self.l1 = torch.nn.Linear(len(labels), 100)
        self.l2 = torch.nn.Linear(100, 40)
        self.l3 = torch.nn.Linear(40, len(elements))

    def forward(self, x):
        x = torch.tanh(self.l1(x))
        x = torch.tanh(self.l2(x))
        x = self.l3(x)
        return x
    
model = Model_Torch()
model.load_state_dict(torch.load('master_thesis/data/pytorch_state_dict.pt'))
model.eval();

# Setup priors
The inputs to the model are drawn from a fixed set of global galaxy parameters (slope of the IMF & Rate of Type Ia Supernove) and a set of local star parameters.

In [7]:
local_GP = utils.MultipleIndependent(
    [Normal(p[0]*torch.ones(1), p[1]*torch.ones(1)) for p in priors[2:]] +
    [Uniform(torch.tensor([2.0]), torch.tensor([12.8]))],
    validate_args=False)

global_GP = utils.MultipleIndependent(
    [Normal(p[0]*torch.ones(1), p[1]*torch.ones(1)) for p in priors[:2]],
    validate_args=False)

# Simulate data
With the set global priors and for each $N_{\rm stars}$ a set of local priors, we can simulate the data with the Neural Network.

In [61]:
N_stars = 1000
simulations = 1000

stars = local_GP.sample((N_stars,))
global_params = torch.tensor([[-2.3, -2.89]])

stars = torch.cat((global_params.repeat(N_stars, 1), stars), dim=1)

In [ ]:
# ----- Simulate abundances -----
start = t.time()
abundances = model(stars)   
end = t.time()
print(f'Time to create data for {N_stars} stars: {end-start:.3f} s')

Time to create data for 1000 stars: 0.028 s


# SBI

Run the sbi inference with the simulated data and save the output for all simulations.

In [ ]:
alpha_IMF = []
log10_N_Ia = []
simulations = 1000

start = t.time()
for i in tqdm(range(len(abundances))):
    alpha, N_Ia = posterior.sample((simulations,), x=abundances[i], show_progress_bars=False)[:,0:2].T
    alpha_IMF.append(alpha)
    log10_N_Ia.append(N_Ia)
end = t.time()
print(f'Time to run {simulations} simulations for {N_stars} stars: {end-start:.3f} s')

alpha_IMF = np.array(alpha_IMF).ravel()
log10_N_Ia = np.array(log10_N_Ia).ravel()

100%|██████████| 1000/1000 [04:34<00:00,  3.64it/s]

Time to run 1000 simulations for 1000 stars: 274.457 s


In [ ]:
# save the data
np.save('master_thesis/data/alpha_IMF.npy', alpha_IMF)
np.save('master_thesis/data/log10_N_Ia.npy', log10_N_Ia)

# SBI with 5% observation error noise

In [9]:
# ----- Load posterior -----
with open('master_thesis/data/posterior_sbi_w5p-error.pickle', 'rb') as f:
    posterior_noise = pickle.load(f)

In [11]:
alpha_IMF_obs = np.load("master_thesis/data/alpha_IMF_obs.npy")
log10_N_Ia_obs = np.load("master_thesis/data/log10_N_Ia_obs.npy")

In [ ]:
def add_noise(true_abundances):
    # Define observational erorrs
    pc_ab = 5 # percentage error in abundance

    # Jitter true abundances and birth-times by these errors to create mock observational values.
    obs_ab_errors = np.ones_like(true_abundances)*float(pc_ab)/100.
    obs_abundances = norm.rvs(loc=true_abundances,scale=obs_ab_errors)

    return obs_abundances

In [ ]:
alpha_IMF_obs = []
log10_N_Ia_obs = []
simulations = 1000

start = t.time()
for i in tqdm(range(len(abundances))):
    x = add_noise(abundances[i].detach().numpy())
    alpha, N_Ia = posterior_noise.sample((simulations,), x=x, show_progress_bars=False)[:,0:2].T
    alpha_IMF_obs.append(alpha)
    log10_N_Ia_obs.append(N_Ia)
end = t.time()
print(f'Time to run {simulations} simulations for {N_stars} stars: {end-start:.3f} s')

alpha_IMF_obs = np.array(alpha_IMF_obs).ravel()
log10_N_Ia_obs = np.array(log10_N_Ia_obs).ravel()

100%|██████████| 1000/1000 [05:28<00:00,  3.04it/s]

Time to run 1000 simulations for 1000 stars: 328.648 s


In [ ]:
# save the data
np.save('master_thesis/data/alpha_IMF_obs.npy', alpha_IMF_obs)
np.save('master_thesis/data/log10_N_Ia_obs.npy', log10_N_Ia_obs)